In [1]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import shutil
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show
import ee
import geemap
from geemap import ml
import requests
import csv
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import cv2
from geopy.distance import geodesic
from joblib import load
from concurrent.futures import ThreadPoolExecutor, as_completed
import boto3
import io
import fiona
import tempfile
import logging
import csv
import threading
#from keys import *

from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [2]:
lock = threading.Lock()  # Create a lock for file access
text_lock = threading.Lock()
output_lock = threading.Lock()
batch_lock = threading.Lock()

In [3]:
def mask_s2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    return image.updateMask(mask).divide(10000)

# Define the NDVI calculation function
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Define the clipping function
def clipper(image,ee_geometry):
    return image.clip(ee_geometry)


def display_sentinel2image(Map, ee_geometry, start_date, end_date, bands, ee_classifier):

    roi = ee_geometry
    

    user_image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 2)) \
        .map(mask_s2clouds) \
        .map(add_ndvi) \
        .map(lambda img: clipper(img, ee_geometry)) \
        .median().select(bands)

    #Map.add_layer(user_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'user_image')

    return user_image


def are_close(coord1, coord2, threshold):
    return geodesic(coord1, coord2).meters <= threshold

# Clustering function
def cluster_coordinates(coordinates, threshold):
    clusters = []
    for coord in coordinates:
        found_cluster = False
        for cluster in clusters:
            for c in cluster:
                if are_close(coord, c, threshold):
                    cluster.append(coord)
                    found_cluster = True
                    break
            if found_cluster:
                break
        if not found_cluster:
            clusters.append([coord])
    return clusters

def find_center(coordinates):
    if not coordinates:
        return None  
    x_coords, y_coords = zip(*coordinates)

    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)

    return (center_x, center_y)

def remove_isolated_pixels(coordinates, threshold_distance):
    if not coordinates:
        return [] 
    
    isolated_pixels = set()

    for i, coord in enumerate(coordinates):

        if coord in isolated_pixels:
            continue

        isolated = True

        for j, other_coord in enumerate(coordinates):
            if i != j: 
                distance = geodesic(coord, other_coord).meters
                if distance <= threshold_distance:
                    isolated = False 
                    break 

        if isolated:
            isolated_pixels.add(coord) 

    filtered_coordinates = [coord for coord in coordinates if coord not in isolated_pixels]

    return filtered_coordinates

def write_centers_to_csv(centers, csv_file):
    with output_lock:
        with open(csv_file, 'a', newline='') as file:
            writer = csv.writer(file)
            for center in centers:
                writer.writerow(center)

def write_to_unused_csv(file_name):
    # Check if the CSV file exists
    with lock:
        # Open the CSV file in append mode
        with open(unused_files_csv, mode='a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([file_name])

def process_sentinel_and_detect_red_pixels(shapefile, start_date, end_date, feature_names, rf_classifier, i):
    # Convert shapefile to Earth Engine object
    ee_geometry = geemap.shp_to_ee(shapefile)

    # Display Sentinel-2 image
    sentinel_image = display_sentinel2image(Map, ee_geometry, start_date, end_date, feature_names, None)

    # Export the classified image
    url = sentinel_image.getDownloadURL({
        'name': 'sentinel_image',
        'bands': ['B4', 'B3', 'B2'],
        'scale': 10,
        'region': ee_geometry.geometry(),
        'format': 'GEO_TIFF'
    })
    
    response = requests.get(url)
    file_like_object = io.BytesIO(response.content)

    with rasterio.open(file_like_object) as src:
        src = src.read()
        src = src * 2.5
        #print('Original image:')
        #show(src)

    file_like_object.seek(0)

    with rasterio.open(file_like_object) as src:
        width = src.width
        height = src.height
        transforms = src.transform

        pixel_data = []
        for row in range(height):
            for col in range(width):
                b4_value = src.read(1, window=((row, row+1), (col, col+1)))
                b3_value = src.read(2, window=((row, row+1), (col, col+1)))
                b2_value = src.read(3, window=((row, row+1), (col, col+1)))
                lon, lat = src.xy(row, col)
                pixel_data.append([b4_value[0][0], b3_value[0][0], b2_value[0][0], lat, lon])

    df = pd.DataFrame(pixel_data, columns=['B4', 'B3', 'B2', 'Latitude', 'Longitude'])

    X = df[['B4', 'B3', 'B2']]
    y_pred = rf_classifier.predict(X)
    df['landcover'] = y_pred

    output_memfile = io.BytesIO()

    with rasterio.MemoryFile() as memfile:
        with memfile.open(driver='GTiff', width=width, height=height, count=4,
                          dtype=np.uint8, crs='EPSG:4326', transform=transforms) as dst:
            dst.write(df['B4'].values.reshape(height, width), 1)
            dst.write(df['B3'].values.reshape(height, width), 2)
            dst.write(df['B2'].values.reshape(height, width), 3)
            dst.write(df['landcover'].values.reshape(height, width), 4)

        output_memfile = memfile.read()

    with rasterio.MemoryFile(output_memfile) as memfile:
        with memfile.open() as src:
            mask_band = src.read(4)

    new_image = np.zeros((mask_band.shape[0], mask_band.shape[1], 3), dtype=np.uint8)
    for value in [0, 1, 2]:
        if value == 0:
            new_image[mask_band == value] = [255, 0, 0] 
        else:
            new_image[mask_band == value] = [0, 0, 0] 

   # plt.imshow(new_image)
    #plt.axis('off')
    #plt.show()

    kernel_closed = np.ones((3, 3), np.uint8)
    image_closed = cv2.morphologyEx(new_image, cv2.MORPH_CLOSE, kernel_closed)
    
    #print('Closed image:')
    #plt.imshow(image_closed)
    #plt.axis('off')
    #plt.show()

    red_points = np.argwhere((image_closed == [255, 0, 0]).all(axis=2))
    red_coordinates = [transforms * (point[1], point[0]) for point in red_points]

    if len(red_coordinates) > 0:
        red_coordinates = remove_isolated_pixels(red_coordinates, 10)
        clusters = cluster_coordinates(red_coordinates, 25)
        centers = [find_center(cluster) for cluster in clusters]
        centers = [(center[1], center[0]) for center in centers]
    
        if len(centers) > 7:
            centers = [centers[0]]
       
        print('Red pixel clusters:')
        print(centers)
        write_centers_to_csv(centers, csv_file)

    else:
        print('No red points found')

def process_shapefile(shapefile_path, start_date, end_date, feature_names, rf_classifier, index):
    try:
        process_sentinel_and_detect_red_pixels(shapefile_path, start_date, end_date, feature_names, rf_classifier, index)
    except Exception as e:
        print(f"Error processing file {shapefile_path}: {e}")
        write_to_unused_csv(shapefile_path)

def process_batch(batch, batch_number, text_file, batch_lock):
    with batch_lock:
        print(f'Processing batch number: {batch_number}')
        with open(text_file, 'w') as f:
            f.write(f'{batch_number}')
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = [
            executor.submit(process_shapefile, file, start_date, end_date, feature_names, rf_classifier, 0)
            for file in batch if file.endswith('.shp')
        ]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f'Error processing file: {e}')

In [13]:
rf_classifier = load('../../models/RF_model_v3.pkl')

In [4]:
feature_names = ['B4', 'B3', 'B2']
#start date should be 1st june 2023
start_date = '2023-06-01'
end_date = '2023-06-15'

In [5]:
csv_file = '../Data/coordinate_csvs_2/centers_0_1000.csv'
unused_files_csv = '../Data/error_csvs_2/not_processed_0_1000_2.csv'
text_file = '../Data/latest_processed/latest_processed_0_1000.txt'

In [6]:
ee.Initialize()
Map = geemap.Map()

In [7]:
import os

# Define the directory containing the shapefiles
shapefile_directory = '../../grids_final_pak_final_version'

# Get a list of files in the directory
files = os.listdir(shapefile_directory)
# Check if there are any files in the directory
if files:
    first_file = files[0]
    print(f"The first file in the directory is: {first_file}")

The first file in the directory is: grid_0.shp


In [8]:
folder_path = '../../grids_final_pak_final_version'
shapefiles = []

# Walk through the directory and get file paths
for root, dirs, files in os.walk(folder_path):
    for file in files:
        shapefiles.append(os.path.join(root, file))

print(f"Total files found: {len(shapefiles)}")

Total files found: 1074872


In [9]:
shapefiles = shapefiles[:1074800]
batch_size = 200

In [10]:
shapefile_batches = [shapefiles[i:i + batch_size] for i in range(0, len(shapefiles), batch_size)]
len(shapefile_batches)

5374

In [11]:
shapefile_batches = shapefile_batches[748:1000]
len(shapefile_batches)

252

In [12]:
#shapefile_batches[-1] = shapefile_batches[-1][1:]
shapefile_batches[100]

['../../grids_final_pak_final_version/grid_84791.dbf',
 '../../grids_final_pak_final_version/grid_84792.shp',
 '../../grids_final_pak_final_version/grid_84792.dbf',
 '../../grids_final_pak_final_version/grid_84793.shp',
 '../../grids_final_pak_final_version/grid_84793.dbf',
 '../../grids_final_pak_final_version/grid_84794.shp',
 '../../grids_final_pak_final_version/grid_84794.dbf',
 '../../grids_final_pak_final_version/grid_84795.shp',
 '../../grids_final_pak_final_version/grid_84795.dbf',
 '../../grids_final_pak_final_version/grid_84796.shp',
 '../../grids_final_pak_final_version/grid_84796.dbf',
 '../../grids_final_pak_final_version/grid_84797.shp',
 '../../grids_final_pak_final_version/grid_84797.dbf',
 '../../grids_final_pak_final_version/grid_84798.shp',
 '../../grids_final_pak_final_version/grid_84798.dbf',
 '../../grids_final_pak_final_version/grid_84799.shp',
 '../../grids_final_pak_final_version/grid_84799.dbf',
 '../../grids_final_pak_final_version/grid_84800.shp',
 '../../gr

In [14]:
for batch_number, batch in enumerate(shapefile_batches):
    process_batch(batch, batch_number + 748, text_file, batch_lock)

Processing batch number: 748
Error processing file ../../grids_final_pak_final_version/grid_74857.shp: '/vsipythonfilelike/d7009b58-3568-4d90-9377-f80fbf5141b7/d7009b58-3568-4d90-9377-f80fbf5141b7' not recognized as a supported file format.
Error processing file ../../grids_final_pak_final_version/grid_74797.shp: '/vsipythonfilelike/138c4239-0161-409f-9b91-67ef37c10188/138c4239-0161-409f-9b91-67ef37c10188' not recognized as a supported file format.
Error processing file ../../grids_final_pak_final_version/grid_74853.shp: '/vsipythonfilelike/093692ef-c486-4c78-8211-3c32a5d6db1a/093692ef-c486-4c78-8211-3c32a5d6db1a' not recognized as a supported file format.
Error processing file ../../grids_final_pak_final_version/grid_74792.shp: '/vsipythonfilelike/1d046ad8-b534-426d-b0f0-576cdac3da80/1d046ad8-b534-426d-b0f0-576cdac3da80' not recognized as a supported file format.
Error processing file ../../grids_final_pak_final_version/grid_74820.shp: '/vsipythonfilelike/9c8827e1-5aab-4256-ae67-c15e4